# Covariance Matrix from Betas

In [5]:
β      = [0.9;1.1]
StdRes = [10;5]
VarRm  = 4^2
Σ      = diagm(StdRes.^2)                  #correlation matrix -> covariance matrix 
println(round(Σ,3))

CovR = β*β'*VarRm + Σ
println("Covariance matrix calculated from betas: ")
println(round(CovR,2))

[100.0 0.0
 0.0 25.0]
Covariance matrix calculated from betas: 
[112.96 15.84
 15.84 44.36]


# Estimating a Single-Index Model

In [6]:
xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
x    = xx[1]
Rme  = x[:,2]                #market excess return
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1;7;13;19;25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                    #no. obs and  no. test assets
#-------------------------------------------------------------------------

c      = ones(T,1)                 #a vector with ones
x      = [c Rme]                   #regressors
β      = fill(NaN,n)
VarRes = fill(NaN,n)
for i = 1:n
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    β[i]      = b_i[2]
    VarRes[i] = var(residual)
end    
println("\nOLS coefficients, regressing Re on constant and Rme")
println(round(β,3)) 
#-------------------------------------------------------------------------

Σ     = diagm(VarRes)  
VarRm = var(Rme)
CovR  = β*β'*VarRm + Σ
println("\nCovariance matrix calculated from betas: ")
println(round(CovR,2))

println("\nCovariance matrix calculated from data: ")
println(round(cov(Re),2))


OLS coefficients, regressing Re on constant and Rme
[1.341,1.169,0.994,0.943,0.849]

Covariance matrix calculated from betas: 
[73.48 33.23 28.27 26.8 24.15
 33.23 37.37 24.64 23.36 21.05
 28.27 24.64 26.8 19.87 17.91
 26.8 23.36 19.87 25.2 16.97
 24.15 21.05 17.91 16.97 25.33]

Covariance matrix calculated from data: 
[73.48 43.02 29.47 25.01 20.19
 43.02 37.37 29.07 25.14 21.41
 29.47 29.07 26.8 23.63 20.98
 25.01 25.14 23.63 25.2 21.53
 20.19 21.41 20.98 21.53 25.33]


# A Multi-Index Model

In [7]:
x      = [c Rme RSMB RHML]               #regressors
K      = size(x,2) - 1 
b      = fill(NaN,(K,n))
VarRes = fill(NaN,n)
for i = 1:n
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    b[:,i]    = b_i[2:end]
    VarRes[i] = var(residual)
end    
println("\nOLS coefficients, regressing Re on [constant, Rme, RSMB, RHML] ")
println(round(b,3)) 
#-------------------------------------------------------------------------

Omega = cov(x[:,2:end])      #covariance matrix of factors
CovR = fill(NaN,(n,n))
for i = 1:n    
    for j = 1:n
        CovR[i,j] = (b[:,i]'Omega*b[:,j])[1]
        if i == j
            CovR[i,i] = CovR[i,i] + VarRes[i]
        end
    end    
end

println("\nCovariance matrix calculated from betas: ")
println(round(CovR,2))

println("\nCovariance matrix calculated from data: ")
println(round(cov(Re),2))


OLS coefficients, regressing Re on [constant, Rme, RSMB, RHML] 
[1.07 1.08 1.035 1.056 1.041
 1.264 0.768 0.437 0.153 -0.088
 -0.278 0.16 0.487 0.603 0.77]

Covariance matrix calculated from betas: 
[73.48 41.85 31.05 25.45 18.94
 41.85 37.37 27.38 24.14 20.15
 31.05 27.38 26.8 22.24 20.11
 25.45 24.14 22.24 25.2 20.72
 18.94 20.15 20.11 20.72 25.33]

Covariance matrix calculated from data: 
[73.48 43.02 29.47 25.01 20.19
 43.02 37.37 29.07 25.14 21.41
 29.47 29.07 26.8 23.63 20.98
 25.01 25.14 23.63 25.2 21.53
 20.19 21.41 20.98 21.53 25.33]
